In [13]:
#データの読込
import numpy as np
import pandas as pd
import codecs

with codecs.open("input_6man.csv","r","Shift-JIS","ignore") as file:
    df = pd.read_csv(file)
with codecs.open("input_6man.csv","r","Shift-JIS","ignore") as file2:
    df2 = pd.read_csv(file2)
df

,player_id,frag_starting,play_time,points,success_3p,try_3p,success_2p,try_2p,dunk,success_ft,...,block,block_receive,fast_break_points,points_from_turn_over,2p_inside_points,second_chance_points,frag_6man,period,position,probability_6man
0,8446,120,71985,355,25,87,116,240,0,48,...,3,9,12,30,86,51,0,186,PG,0.193548
1,8447,117,69288,432,60,187,102,217,0,48,...,12,14,53,65,160,34,0,194,PG,0.193548
2,8448,147,75513,378,43,126,110,208,1,29,...,6,14,50,75,184,48,0,195,SF,0.048387
3,8449,47,35008,178,46,127,18,43,0,4,...,1,5,7,8,10,8,0,121,PG/SG,0.145161
4,8450,177,105495,549,63,228,121,282,0,118,...,15,37,58,104,216,49,0,239,SF,0.016393
5,8451,59,30520,110,11,30,32,70,4,13,...,1,6,4,7,64,15,0,144,SF,0.096774
6,8452,164,106050,820,50,139,269,481,57,132,...,56,30,64,91,456,138,0,237,SF/PF,0.112903
7,8454,95,56338,176,23,95,34,94,0,39,...,3,11,10,28,56,2,0,173,PG,0.138462
8,8455,35,20783,56,1,15,26,57,0,1,...,1,1,2,4,16,2,0,77,SG,0.048387
9,8456,71,43121,247,19,60,69,189,0,52,...,2,13,10,19,80,21,0,114,PG/SG,0.023256


In [14]:
#プレイタイム上位半分のデータを選択
median = df["play_time"].median()
df = df.loc[df["play_time"] >= median]

#ポジションデータを一旦消去
del df["position"]

#データの標準化
df_st = df.apply(lambda x: (x-x.mean())/x.std(), axis=0).fillna(0)

#選手IDと6manのデータの変更
df_st["player_id"] = df["player_id"]
df_st["frag_6man"] = df["frag_6man"]
df_st["position"] = df2["position"]

#行番号の振り直し
df_st = df_st.reset_index(drop = True)

In [15]:
#csv出力
df_st.to_csv("coloring_6man.csv")

In [16]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

with codecs.open("coloring_6man.csv","r","Shift-JIS","ignore") as file:
    playerdf = pd.read_csv(file)
    
#選手IDを削除
del playerdf["player_id"]
del playerdf["frag_6man"]
del playerdf["position"]
del playerdf["probability_6man"]

#dfをarrayに変換
player = playerdf.as_matrix()

#行番号を削除
player = np.delete(player,0,1)

In [18]:
#PCAで次元削減
pca = PCA(n_components=2)
player_r = pca.fit_transform(player)

#probability_6manのデータフレームを作成
df_6man = df_st["probability_6man"].rank()

# 結果を散布図にプロット
plt.figure()
for i in range(0, len(df_6man )):
    if df_6man[i] <= len(df_6man ) * 1/2 :
        plt.scatter(player_r[i, 0], player_r[i, 1], c= "red", alpha = 0.5)
    else:
        plt.scatter(player_r[i, 0], player_r[i, 1], c= "blue", alpha = 0.5)
        
"""
for i in range(0, len(df_6man )):
    if df_6man[i] <= len(df_6man ) * 1/7 :
        plt.scatter(player_r[i, 0], player_r[i, 1], c= "red")
    elif df_6man[i] <= len(df_6man ) * 2/7:
        plt.scatter(player_r[i, 0], player_r[i, 1], c= "yellow")
    elif df_6man[i] <= len(df_6man ) * 3/7:
        plt.scatter(player_r[i, 0], player_r[i, 1], c= "green")
    elif df_6man[i] <= len(df_6man ) * 4/7:
        plt.scatter(player_r[i, 0], player_r[i, 1], c= "skyblue")
    elif df_6man[i] <= len(df_6man ) * 5/7:
        plt.scatter(player_r[i, 0], player_r[i, 1], c= "blue")
    elif df_6man[i] <= len(df_6man ) * 6/7:
        plt.scatter(player_r[i, 0], player_r[i, 1], c= "darkblue")
    else:
        plt.scatter(player_r[i, 0], player_r[i, 1], c= "purple")
"""

plt.title("coloring_by_6man")
        
#画像として保存
plt.savefig("coloring_by_6man.png")